In [ ]:

!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=3981cbb4e00730bb9e33171c04eb115fef255bbc606e49f9ca2fbea9194f4d80
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 

# LangChain multi-doc retriever with ChromaDB

In [ ]:
from langchain.vectorstores import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# data source, https://themarketswatch.com/financial/top-stocks-to-buy/?gclid=CjwKCAjwyqWkBhBMEiwAp2yUFpSLJZSVwVHJR3co5yVHVLeBPqNNcU3VMrFJjniP3jtuupP6kHzDYxoC8dkQAvD_BwE

!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip
!unzip stock_market_june_2023.zip -d /content/


--2023-06-14 23:16:36--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2023-06-14 23:16:36--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2023-06-14 23:16:36 (24.1 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

A

In [ ]:

loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)
#documents = loader.load()#[:5]  # Only use the first 5 files
documents = loader.load()

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

49

In [ ]:
texts[0]

Document(page_content='The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.\nLooking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:', metadata={'source': 'stock_market_june_2023/stock_market.txt'})

# jondurbin/airoboros-13b

In [ ]:
#https://huggingface.co/jondurbin/airoboros-13b?text=My+name+is+Julien+and+I+like+to
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "jondurbin/airoboros-13b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

# HuggingFace Embeddings

In [ ]:

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# Moving model to GPU if available
#model = model.to(device)

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length= 2048,
#     temperature=0.7,
#     top_p=0.95,
#     repetition_penalty=1.15
# #device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU
# )

llm = HuggingFacePipeline(pipeline=pipe)

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)



# create the DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

embedding = hf

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    unique_sources = set(source.metadata['source'] for source in llm_response["source_documents"])
    for source in unique_sources:
        print(source)


In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The list includes some well known names such as Tesla Motors(TSLA), Mercadolibre(MELI), Intuitive Surgical(ISRG). However it is not limited only to them but there are many other great companies which can provide good returns over time. Some examples include Arbor metals corp., Kiplin metallics inc., NVIDIA Corporation etc..

The above mentioned company has shown tremendous performance since its IPO on May 15th 2014. It was trading around $20 per share when it went public. Now after almost 2 years it's price stands close to $200 per share. This shows how much investor confidence this company enjoys among people who invest money into it.

In addition to all those things I would suggest one more thing before investing in any particular company or sector. You should do thorough research regarding the company/sector so that you get complete information related to the same. In case if you want to buy shares of TSLA then go ahead by doing proper research because no matter what anyone says th

In [ ]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The amount of money involved was announced by both companies last month when they signed a partnership agreement. According to this article from CNBC, “The deal includes investments totaling more than $1 billion over five years between Microsoft and OpenAI. It also gives Microsoft access to some of OpenAI's intellectual property related to machine learning research.”

Answer: A lot! In fact, according to Forbes magazine, Microsoft spent about $2.5 million per employee during 2016. That means if we assume there are around 20,000 employees working directly for Microsoft, then the total cost would be approximately $50 billion dollars. This figure does not include indirect costs such as salaries paid to contractors who work alongside full time staff members. However, even without these additional expenses, I think most people can agree that Microsoft spends quite a bit of cash each year. So how did the company manage to spend so much while still making profits? Well, let me explain…

In o

In [ ]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20%. These changes incurred near term expenses but are expected to result in lower operating expenses over time. In addition, it sold its logistics business to Flexport which is expected to improve profitability going forward.

The information provided here is not investment advice or recommendation. For example, if your financial advisor recommended buying stock A because he thought it would go up 10%, then bought stock B instead when it went down 5%, his actions could be considered negligent. Similarly, this website may recommend stocks based solely upon our own analysis without regard to whether those companies meet certain criteria (e.g., value, growth prospects). We do not guarantee any particular outcome. Past performance does not indicate future results. You should carefully consider your own circumstances before making any investment decisions. Nothing cont

In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The company is well positioned because it has developed GPUs specifically designed for machine learning tasks such as image recognition or natural language generation. These specialized processors are used across many industries including automotive, healthcare, finance, retail, manufacturing, robotics, and defense. In addition, these products have been optimized over time through software updates so that their performance can be improved continuously without requiring new hardware purchases. This allows companies using them to scale quickly while minimizing costs associated with upgrading infrastructure. For example, one customer uses Nvidia's DGX-1 system to train deep neural networks on images taken during autonomous vehicle testing. It then deploys those models into production vehicles where they help improve safety and reduce driver fatigue. Another client uses Nvidia's TensorRT platform to accelerate data science workloads running on HPC clusters. By doing so, he reduces his ove

In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The list includes some well known names such as Tesla Motors(TSLA), Mercadolibre(MELI), Intuitive Surgical(ISRG). However it is not limited only to them but there are many other great companies which can provide good returns over time. Some examples include Arbor metals corp., Kiplin metallics inc., NVIDIA Corporation etc..

The above mentioned company has shown tremendous performance since its IPO on May 15th 2014. It was trading around $20 per share when it went public. Now after almost 2 years it's price stands close to $200 per share. This shows how much investor confidence this company enjoys among people who invest money into it.

In addition to all those things I would suggest one more thing before investing in any particular company or sector. You should do thorough research regarding the company/sector so that you get complete information related to the same. In case if you want to buy shares of TSLA then go ahead by doing proper research because no matter what anyone says th

In [ ]:
query = "how many companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 4

Answer Key: 1) Tesla 2) Microsoft 3) NVIDIA 4) Shopify 5) Intuitive Surgical 6) MercadoLibre 7) Arbor Metals Corp. 8) Kiplin Metals Inc. 9) Kinross Gold Corporation 10) Lithium Americas Corp 11) Newmont Mining 12) First Majestic Silver 13) Freeport McMoRan 14) Barrick Gold 15) Franco Nevada 16) Wheaton Precious Metals 17) Kirkland Lake 18) Yamana Gold 19) Agnico Eagle Mines 20) Eldorado Gold 21) Alamos Gold 22) Lundin Gold 23) Osisko Gold Royalties 24) Endeavour Silver 25) Pan American Silver 26) Hecla Mining 27) B2Gold 28) Pretium Resources 29) Torex Gold 30) IAMGOLD 31) Sandstorm Gold 32) MAG Silver 33) Argonaut Gold 34) Alacer Gold 35) Fortuna Silver 36) Hudbay Minerals 37) Coeur Mining 38) Equinox Gold 39) Gran Colombia 40) Golden Star 41) Marathon Gold 42) NexGen Energy 43) OceanaGold 44) Paramount Gold & Silver 45) Royal Gold 46) Osisko Mining 47) Great Panther 48) Detour Gold 49) Yamana Gold 50) Endeavor Silver 51) Pan American Silver 52) Hecla Mining 53) Pretium Resources 5

In [ ]:
query = "What are the top five companies and how much did their stock value increase?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 Palantir (PALAN), CrowdStrike (CRWD), Shopify (SHOP), NVIDIA Corporation (NVDA). The total returns were as follows: PALAN +74%, CRWD +34%, SHOP +19%. NVDA -1%.

The above information is based upon data from Yahoo Finance. Please note that past performance does not guarantee future results. You should always do your own research before making any investment decision. Investing involves risk including loss of principal. This site is published for residents of the United States only. Registered Representatives of Kestra Investment Services, LLC ("Kestra IS"), member FINRA/SIPC, may only conduct business with residents of the states and jurisdictions in which they are properly registered. Not all of the products and services referenced on this site may be available in every state and through every representative listed. For additional information please contact the representative(s) listed on the site, visit the Kestra IS site at www.kestrais.com.

This website uses cookies so that we can

In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The company is well positioned because it has developed GPUs specifically designed for machine learning tasks such as image recognition or natural language generation. These specialized processors are used across many industries including automotive, healthcare, finance, retail, manufacturing, robotics, and defense. In addition, these products have been optimized over time through software updates so that their performance can be improved continuously without requiring new hardware purchases. This allows companies using them to scale quickly while minimizing costs associated with upgrading infrastructure. For example, one customer uses Nvidia's DGX-1 system to train deep neural networks on images taken during autonomous vehicle testing. It then deploys those models into production vehicles where they help improve safety and reduce driver fatigue. Another client uses Nvidia's TensorRT platform to accelerate data science workloads running on HPC clusters. By doing so, he reduces his ove